#Visualize Images

In [1]:
import ee
import os
import random
ee.Authenticate()
ee.Initialize(project = 'tony-1122')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


#***To visualize RGB***

In [ ]:
# Initialize Google Cloud Storage client
folder = 'indo'
from google.cloud import storage
client = storage.Client()
# Specify your bucket name and the prefix (folder path in the bucket)
bucket_name = 'mangrove_clay'
prefix = f'sentinel_2023/sentinel_patches_{folder}/'

bucket = client.get_bucket(bucket_name)

# List all files in the specified prefix (folder)
blobs = bucket.list_blobs(prefix=prefix)

# Create a list of file paths
gcs_paths = ['gs://' + bucket_name + '/' + blob.name for blob in blobs if blob.name.endswith('.tif')]

# Check the number of patches found
print(f"Found {len(gcs_paths)} patches.")

Found 473 patches.


#***To visualize inference (probability)***

In [62]:
# Initialize Google Cloud Storage client
country = 'brunei'
year = '2024'

from google.cloud import storage
client = storage.Client()
# Specify your bucket name and the prefix (folder path in the bucket)
bucket_name = 'mangrove_clay'

prefix = f'inference_remain/COG_{country}_{year}/'

bucket = client.get_bucket(bucket_name)

# List all files in the specified prefix (folder)
blobs = bucket.list_blobs(prefix=prefix)

# Create a list of file paths
gcs_paths = ['gs://' + bucket_name + '/' + blob.name for blob in blobs if blob.name.endswith('.tif')]

# Check the number of patches found
print(f"Found {len(gcs_paths)} patches.")

Found 7 patches.


In [63]:
import geemap
# Visualization parameters
vis_params = {'bands': ['B2', 'B1', 'B0'], 'min': 400, 'max': 3600, 'gamma': 1}
vis_prob = {'min': 0, 'max': 1, 'palette': ['white', 'yellow', 'orange', 'red', 'purple']}

# Load the patches as Earth Engine images
images = [ee.Image.loadGeoTIFF(gcs_path) for gcs_path in gcs_paths[:2000]]
# boundary = ee.FeatureCollection(f'projects/tony-1122/assets/{country}_boundary')
boundary = ee.FeatureCollection(f'projects/tony-1122/assets/indo_boundary')
# Create a mosaic of all the patches
mosaic_image = ee.ImageCollection(images).mosaic().clip(boundary)

# Create a map object
Map = geemap.Map()
# Set the basemap to satellite
Map.add_basemap('SATELLITE')
# Add the mosaic image to the map
Map.addLayer(mosaic_image, vis_prob, "Mosaic of Patches")
Map.addLayer(boundary)
# Display the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [64]:
# Export the mosaic image to a GEE asset
task = ee.batch.Export.image.toAsset(
    image=mosaic_image,
    description=f'{country}_inference_{year}',  # Name your task
    assetId=f'projects/tony-1122/assets/mangrove_inference_remain/{country}_inference_{year}',  # Replace with your GEE asset ID path
    scale=10,  # Specify the resolution in meters
    region= boundary.geometry().bounds(),  # Define the region to export
    maxPixels=1e13  # Set a large maxPixels if the image is large
)

# Start the export task
task.start()

In [ ]:
batch_size = 2000
for i in range(0, len(gcs_paths), batch_size):
    # Load a batch of images from Google Cloud Storage paths
    images = [ee.Image.loadGeoTIFF(gcs_path) for gcs_path in gcs_paths[i:i+batch_size]]

    # Mosaic and clip the batch
    mosaic_image = ee.ImageCollection(images).mosaic().clip(boundary)

    # Export the mosaic image to a GEE asset
    task = ee.batch.Export.image.toAsset(
        image=mosaic_image,
        description=f'{country}_inference_{year}_{i}',  # Task description with batch index
        assetId=f'projects/tony-1122/assets/mangrove_inference/{country}_inference_{year}_{i}',  # Asset path
        scale=10,  # Specify the resolution in meters
        region=boundary.geometry().bounds(),  # Define the region to export
        maxPixels=1e13  # Set maxPixels for large images
    )

    # Start the export task
    task.start()
